In [2]:
import pandas as pd
from utils import persistence as ps
from urllib3.response import HTTPResponse
from typing import List, Dict, Tuple
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from toolz import partition_all

In [5]:
INPUT_BUCKET = 'dq-data'
HASH_BUCKET = 'dq-hashed'

In [6]:
#load train_set
data = 'train.csv'
filestream = ps.get_file_stream(bucket=INPUT_BUCKET, filename=data)
dtypes = {
    'id': 'int64',
    'qid1': 'int64',
    'qid2': 'int64',
    'question1': 'object',
    'question2': 'object',
    'is_duplicate': 'int64'
}
df = pd.read_csv(#urlpath=s3_in_url, 
                                     #storage_options=s3_options,
                                     filestream,
                                     header=0, 
                                     usecols=dtypes.keys(), 
                                     #names=dtypes.keys(),
                                     skipinitialspace=True,
                                     skip_blank_lines=True,
                                     encoding='utf-8')
df = df.set_index('id')

In [7]:
df.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


### Train-test split

In [8]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['is_duplicate'])
y = df['is_duplicate']
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.33, random_state=42)

In [9]:
del X,y,df

# Feature Extraction

### Tokenizing and preprocessing

In [10]:
from gensim.parsing.preprocessing import preprocess_string
def get_tokens(process='train'):
    if process=='test':
        X = X_test
    else:
        X = X_train
    series = pd.Series(pd.concat([X['question1'], X['question2']]),dtype=str)
    series.dropna()
    for question in series:
        yield preprocess_string(question)

### Word2Vec (fasttext)

In [11]:
ps.get_file(bucket=INPUT_BUCKET, filename='cc.en.300.bin.gz', filepath='/tmp/cc.en.300.bin.gz')

In [12]:
import gzip
import shutil
with gzip.open('/tmp/cc.en.300.bin.gz', 'rb') as f_in:
    with open('/tmp/cc.en.300.bin', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)  

In [13]:
import os
os.remove('/tmp/cc.en.300.bin.gz')
from gensim.models import FastText
model = FastText.load_fasttext_format('/tmp/cc.en.300.bin')

In [14]:
def get_ft_vectors(model, process):
    for tokens in get_tokens(process):
        vectors = []
        for token in tokens:
            try:
                vector = model.wv[token]
            except:
                continue
            vectors.append(vector)
        yield np.array(vectors)

In [15]:
X_ft = np.array([vectors for vectors in get_ft_vectors(model,'train')])
X_ft.shape

(541748,)

In [16]:
# split back into two
X1_ft = X_ft[:len(X_train)]
X2_ft = X_ft[len(X_train):]

In [17]:
del X_ft

In [18]:
#X_train = pd.concat([X_train, pd.Series(X1_ft, name='q1_ft',index=X_train.index), pd.Series(X2_ft, name='q2_ft',index=X_train.index)], axis=1)
#X_train.head()

##### Test set

In [19]:
X_ft_test = np.array([vectors for vectors in get_ft_vectors(model,'test')])
X_ft_test.shape

(266832,)

In [20]:
del model

In [21]:
# split back into two
X1_ft_test = X_ft_test[:len(X_test)]
X2_ft_test = X_ft_test[len(X_test):]

In [22]:
del X_ft_test

In [23]:
#X_test = pd.concat([X_test, pd.Series(X1_ft_test, name='q1_ft',index=X_test.index), pd.Series(X2_ft_test, name='q2_ft',index=X_test.index)], axis=1)
#X_test.head()

### MDS, LLE Embedding

In [39]:
#from sklearn.manifold import MDS, LocallyLinearEmbedding
#def reduce_dim(X, method, dimensions=3):
#    n_jobs = -1
#    n_neighbors = 5
#    if X.shape[0] <= 5:
#        n_neighbors = X.shape[0] - 1
#    if method == 'LLE':
#        embedding = LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=dimensions, random_state=42, n_jobs=n_jobs)
#    elif method == 'MLLE':
#        embedding = LocallyLinearEmbedding(n_components=dimensions, method='modified', random_state=42, n_jobs=n_jobs)
#    elif method == 'Hessian':
#        embedding = LocallyLinearEmbedding(n_components=dimensions, method='hessian', random_state=42, n_jobs=n_jobs)
#    else: #method == 'MDS':
#        embedding = MDS(n_components=dimensions, random_state=42, n_jobs=n_jobs)
#    X_trfmd = embedding.fit_transform(X)
#    return X_trfmd

In [25]:
from megaman.geometry import Geometry

### Pairwise Metrics

In [40]:
from sklearn.metrics.pairwise import linear_kernel, polynomial_kernel, sigmoid_kernel, laplacian_kernel, rbf_kernel
from scipy.spatial.distance import cdist
def compute_pairwise_kernel(pc1, pc2, method='linear'):
    if method=='polynomial':
        return polynomial_kernel(pc1, pc2, 2)
    elif method=='rbf':
        return rbf_kernel(pc1, pc2)
    elif method=='sigmoid':
        return sigmoid_kernel(pc1, pc2)
    elif method=='laplacian':
        return laplacian_kernel(pc1, pc2)
    else:
        return linear_kernel(pc1, pc2)
    
def compute_pairwise_dist(pc1, pc2, method='euclidean'):
    if pc1.size == 0:
        return []
    if pc2.size == 0:
        return []
    return cdist(pc1, pc2, metric=method)

#def compute_red_dim(pc1, pc2, method='MDS'):
#    if pc1.size == 0:
#        if pc2.size > 0:
#            pc = pc2
#            len_pc1 = 0
#        else:
#            return ()
#    elif pc2.size == 0:
#        pc = pc1
#        len_pc1 = pc1.shape[0]
#    else:
#        pc = np.vstack((pc1,pc2))
#        len_pc1 = pc1.shape[0]
#    pc_embd = reduce_dim(pc, method=method, dimensions=3)
#    pc1_embd = pc_embd[:len_pc1]
#    pc2_embd = pc_embd[len_pc1:]
#    return pc1_embd, pc2_embd
        
def assign_pwmetric(df, method='euclidean'):
    #return compute_pairwise_kernel(pc1_embd, pc2_embd, method=method)
    return df.apply(compute_pairwise_dist, method, axis=1)

In [28]:
#X_train['linear_kernel'] = X_train.apply(rowwise_pwkernel, axis=1)
#X_train.head()
#X_train.to_csv('/tmp/X_train.csv')

In [29]:
#import dask.dataframe as dd
#from dask import delayed, compute
#from dask.distributed import Client
#from utils import dask
#client = dask.create_dask_client(num_workers=8)

In [30]:
#dask.perform_dask_test()

In [31]:
def get_q_lengths(X):
    q_meta = []
    for q in X:
        q_meta.append(len(q))
    return q_meta

In [32]:
X1_ft.shape

(270874,)

In [33]:
def split_arrays(X):
    for y in (x for x in X if x.size>0):
        yield np.vsplit(y,len(y))[0]

In [34]:
q_meta = get_q_lengths(X1_ft) + get_q_lengths(X2_ft)

In [35]:
X = np.vstack((np.concatenate([x for x in split_arrays(X1_ft)]), np.concatenate([x for x in split_arrays(X2_ft)])))

In [36]:
X.shape

(541148, 300)

In [37]:
del X1_ft, X2_ft

In [ ]:
import time
from megaman.embedding import (Isomap, LocallyLinearEmbedding,
                               LTSA, SpectralEmbedding)
radius = 1.5
adjacency_method = 'cyflann'
cyflann_kwds = {'index_type':b'kmeans', 'branching':64, 'iterations':20, 'cb_index':0.4}
adjacency_kwds = {'radius':radius, 'cyflann_kwds':cyflann_kwds}
affinity_method = 'gaussian'
affinity_kwds = {'radius':radius}
laplacian_method = 'geometric'
laplacian_kwds = {'scaling_epps':radius}

geom = Geometry(adjacency_method=adjacency_method, adjacency_kwds=adjacency_kwds,
                affinity_method=affinity_method, affinity_kwds=affinity_kwds,
                laplacian_method=laplacian_method, laplacian_kwds=laplacian_kwds)
geom.set_data_matrix(X)

t0 = time.time()
lle = LocallyLinearEmbedding(n_components=3, eigen_solver='arpack',geom=geom)
X_lle = lle.fit_transform(X=X.astype(np.float))
#adjacency_matrix = geom.compute_adjacency_matrix()
t1 = time.time() - t0
print(t1)

In [ ]:
#X_rd =  reduce_dim(X, 'LLE', 3)

In [ ]:
# rebuild X1_rd and X2_rd
X1_list = []
X2_list = []
for len_q1, len_q2 in q_meta:
    q1 = X_rd[:len_q1]
    q2 = X_rd[len_q1:(len_q1+len_q2)]
    X1_list.append(q1)
    X2_list.append(q2)
    X_rd = X_rd[(len_q1+len_q2):]
X1_rd = np.array(X1_list)
X2_rd = np.array(X2_list)

In [ ]:
del X1_list, X2_list, q_meta, X_rd

In [ ]:
jaccard = []
chebyshev = []
braycurtis = []
for q_tuple in q_rd:
    if q_tuple:
        q1_rd, q2_rd = q_tuple
        jaccard.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'jaccard'))
        chebyshev.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'chebyshev'))
        braycurtis.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'braycurtis'))
    else:
        jaccard.append(delayed([]))
        chebyshev.append(delayed([]))
        braycurtis.append(delayed([])) 

In [ ]:
jaccard = compute(*jaccard)

In [ ]:
chebyshev = compute(*chebyshev)

In [ ]:
braycurtis = compute(*braycurtis)

In [ ]:
len(braycurtis)

In [ ]:
#X_train_ddf = X_train_ddf.map_partitions(reduce_dim,'MDS', meta={'q1_ft':np.float32, 'q2_ft':np.float32})
#X_train_ddf['jaccard'] = X_train_ddf.map_partitions(assign_pwmetric,'jaccard', meta=('jaccard',np.float32))
#X_train_ddf['chebyshev'] = X_train_ddf.map_partitions(assign_pwmetric,'chebyshev', meta=('chebyshev',np.float32))
#X_train_ddf['braycurtis'] = X_train_ddf.map_partitions(assign_pwmetric,'braycurtis', meta=('braycurtis',np.float32))
#X_train = X_train_ddf.compute()
#X_train.head()

##### Test set

In [ ]:
X1_ft_test.shape

In [ ]:
# dask delayed
q_rd_test = []
for q1, q2 in zip(X1_ft_test, X2_ft_test):
    q = delayed(compute_red_dim)(q1, q2, 'MDS')
    q_rd_test.append(q)

In [ ]:
q_rd_test = compute(*q_rd_test)

In [ ]:
len(q_rd_test)

In [ ]:
del X1_ft_test, X2_ft_test

In [ ]:
jaccard_test = []
chebyshev_test = []
braycurtis_test = []
for q_tuple in q_rd_test:
    if q_tuple:
        q1_rd, q2_rd = q_tuple
        jaccard_test.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'jaccard'))
        chebyshev_test.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'chebyshev'))
        braycurtis_test.append(delayed(compute_pairwise_dist)(q1_rd, q2_rd, 'braycurtis'))
    else:
        jaccard_test.append(delayed([]))
        chebyshev_test.append(delayed([]))
        braycurtis_test.append(delayed([]))        

In [ ]:
jaccard_test = compute(*jaccard_test)

In [ ]:
chebyshev_test = compute(*chebyshev_test)

In [ ]:
braycurtis_test = compute(*braycurtis_test)

#### Graph Kernel estimation

In [ ]:
#! pip install grakel-dev

In [ ]:
from grakel import GraphKernel
from grakel import Graph
def estimate_grakel(pc1, pc2):
    sp_kernel = GraphKernel(kernel={"name": "shortest_path"})
    sp_kernel.fit_transform([Graph(pc1)])
    return sp_kernel.transform([Graph(pc2)])

In [ ]:
def rowwise_grakel(row):
    pc1 = row['q1_ft']
    pc2 = row['q2_ft']
    pc1_embd, pc2_embd = embed_pointclouds(pc1, pc2, method='LLE', dimensions=3)
    return estimate_grakel(pc1_embd, pc2_embd)

In [ ]:
X_train['grakel'] = X_train.apply(rowwise_grakel, axis=1)
X_train.head()

In [ ]:
pc1 = X_train.iloc[0]['q1_ft']
pc2 = X_train.iloc[0]['q2_ft']
pc1_embd, pc2_embd = embed_pointclouds(pc1, pc2, method='LLE', dimensions=3)
print(len(pc1))
print(pc1_embd.shape)
print(len(pc2))
print(pc2_embd)

### TFIDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
pass_through = lambda x:x
tfidf = TfidfVectorizer(analyzer=pass_through)
X_trfmd = tfidf.fit_transform(get_tokens('train'))

In [ ]:
X_trfmd

In [ ]:
# dimension reduction using SVD
from sklearn.decomposition import TruncatedSVD
import time
start = time.time()
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
X_svd = svd.fit_transform(X_trfmd)
end =  time.time()
print('created SVD transform in time {}'.format(end-start))

In [ ]:
X_svd.shape

In [ ]:
# split back into two
X1 = X_svd[:len(X_train), :]
X2 = X_svd[len(X_train):, :]

##### Test set

In [ ]:
X_test_trfmd = tfidf.transform(get_tokens('test'))

In [ ]:
X_test_trfmd

In [ ]:
# dimension reduction using SVD
start = time.time()
X_test_svd = svd.transform(X_test_trfmd)
end =  time.time()
print('created SVD transform in time {}'.format(end-start))

In [ ]:
# split back into two
X1_test = X_test_svd[:len(X_test), :]
X2_test = X_test_svd[len(X_test):, :]

In [ ]:
# build complete feature dataframe
X_test_temp = pd.concat([pd.DataFrame(X1_test, columns=['q1_'+str(i) for i in range(X1_test.shape[1])], index=X_test.index), 
                    pd.DataFrame(X2_test, columns=['q2_'+str(i) for i in range(X2_test.shape[1])], index=X_test.index)], axis=1)
X_test_temp.head()

### Fuzzy-wuzzy

In [ ]:
# difference in text size
compute_size_diff = lambda row: abs(len(str(row['question1'])) - len(str(row['question2'])))
X_train['size_diff'] = X_train.apply(compute_size_diff, axis=1)
X_train.head()

In [ ]:
from fuzzywuzzy import fuzz

In [ ]:
# ratio
compute_ratio = lambda row: fuzz.ratio(str(row['question1']), str(row['question2']))
X_train['ratio'] = X_train.apply(compute_ratio, axis=1)
X_train.head()

In [ ]:
# partial ratio
compute_partial_ratio = lambda row: fuzz.partial_ratio(str(row['question1']), str(row['question2']))
X_train['partial_ratio'] = X_train.apply(compute_partial_ratio, axis=1)
X_train.head()

In [ ]:
# token_sort_ratio
compute_token_sort_ratio = lambda row: fuzz.token_sort_ratio(str(row['question1']), str(row['question2']))
X_train['token_sort_ratio'] = X_train.apply(compute_token_sort_ratio, axis=1)
X_train.head()

In [ ]:
# token_set_ratio
compute_token_set_ratio = lambda row: fuzz.token_set_ratio(str(row['question1']), str(row['question2']))
X_train['token_set_ratio'] = X_train.apply(compute_token_set_ratio, axis=1)
X_train.head()

In [ ]:
# build complete feature dataframe
X_train_temp = pd.concat([pd.DataFrame(X1, columns=['q1_'+str(i) for i in range(X1.shape[1])], index=X_train.index), 
                     pd.DataFrame(X2, columns=['q2_'+str(i) for i in range(X2.shape[1])], index=X_train.index)], axis=1)
X_train_temp.head()

In [ ]:
X_train = pd.concat([X_train_temp, X_train], axis=1)
del X_train_temp
X_train = X_train.drop(columns=['qid1', 'qid2','question1','question2'])
X_train.head()

In [ ]:
X_train.info()

##### Test set

In [ ]:
# difference in text size
X_test['size_diff'] = X_test.apply(compute_size_diff, axis=1)
X_test.head()

In [ ]:
# ratio
X_test['ratio'] = X_test.apply(compute_ratio, axis=1)
X_test.head()

In [ ]:
# partial ratio
X_test['partial_ratio'] = X_test.apply(compute_partial_ratio, axis=1)
X_test.head()

In [ ]:
# token_sort_ratio
X_test['token_sort_ratio'] = X_test.apply(compute_token_sort_ratio, axis=1)
X_test.head()

In [ ]:
# token_set_ratio
X_test['token_set_ratio'] = X_test.apply(compute_token_set_ratio, axis=1)
X_test.head()

In [ ]:
X_test = pd.concat([X_test_temp, X_test], axis=1)
del X_test_temp
X_test = X_test.drop(columns=['question1','question2', 'qid1', 'qid2'])
X_test.head()

# Modeling

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
logr_model = LogisticRegression(random_state=42)
param_grid = {'C': np.logspace(-2, 7, 10),
             #'penalty': ['l1','l2'],
             'tol': np.logspace(-5, -1, 5),
             #'solver': ['lbfgs']
             #'max_iter': np.linspace(10, 1000, 10)
             }
logr_cv = RandomizedSearchCV(logr_model, param_distributions=param_grid, cv=5, n_jobs=-1)
logr_cv.fit(X_train, y_train)

In [ ]:
logr_cv.best_params_

In [ ]:
logr_model = LogisticRegression(#solver=logr_cv.best_params_['solver'], 
                                random_state=42, 
                                C=logr_cv.best_params_['C'], 
                                tol=logr_cv.best_params_['tol'], 
                                #max_iter=logr_cv.best_params_['max_iter'], 
                                n_jobs=-1)
logr_model.fit(X_train, y_train)

In [ ]:
logr_pred = logr_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
logr_acc_score = accuracy_score(y_test, logr_pred)
logr_prec_score = precision_score(y_test, logr_pred)
logr_rec_score = recall_score(y_test, logr_pred)
print('Logistic Regression')
print('accuracy score : {}'.format(logr_acc_score))
print('precision score : {}'.format(logr_prec_score))
print('recall score : {}'.format(logr_rec_score))